# lab 6 - wide and deep

In [49]:
import pandas as pd
import numpy as np

data_path = '../../data/attrition.csv'


In [60]:
df = pd.read_csv(data_path)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


There are 35 features in total in the dataset about. 
Let's focus on a few of them:
- Age
- Attrition
- Department
- DistanceFromHome
- Education
- EduacationField
- EnvironmentSatisfaction
- Gender
- JobSatisfaction
- MaritalStatus
- MonthlyIncome
- OverTime
- PerformanceRating
- RelationshipSatisfaction
- TotalWorkingYears
- YearsAtCompany
- YearsSinceLastPromotion

In this lab, we will use attrition as our label, to try to predict the attrition status accroding to other attributes. 


In [62]:
to_keep = {'Age', 'Attrition', 'Department','DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender', 'JobSatisfaction', 'MaritalStatus',
           'MonthlyIncome', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction','TotalWorkingYears','YearsAtCompany', 'YearsSinceLastPromotion'}
to_drop = set(df.columns)-to_keep
df.drop(to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 17 columns):
Age                         1470 non-null int64
Attrition                   1470 non-null object
Department                  1470 non-null object
DistanceFromHome            1470 non-null int64
Education                   1470 non-null int64
EducationField              1470 non-null object
EnvironmentSatisfaction     1470 non-null int64
Gender                      1470 non-null object
JobSatisfaction             1470 non-null int64
MaritalStatus               1470 non-null object
MonthlyIncome               1470 non-null int64
OverTime                    1470 non-null object
PerformanceRating           1470 non-null int64
RelationshipSatisfaction    1470 non-null int64
TotalWorkingYears           1470 non-null int64
YearsAtCompany              1470 non-null int64
YearsSinceLastPromotion     1470 non-null int64
dtypes: int64(11), object(6)
memory usage: 195.3+ KB


It's good that we don't have any null value. 

Let's converet the categorical features (Attrition, Department, EducationField, Gender, MaritalStatus and Overtime) to ints because we'll want Keras to one-hot-encode them later. 

In [63]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

categorical_features = {'Attrition', 'Department', 'EducationField', 'Gender','MaritalStatus','OverTime'}
encoders = dict()
for atr in categorical_features:
    encoders[atr] = LabelEncoder()
    df[atr] = encoders[atr].fit_transform(df[atr] )

Then, let's scale the numeric features. 

In [64]:
numeric_features = set(df.columns) - categorical_features
for atr in numeric_features:
    df[atr] = df[atr].astype(np.float)    
    ss = StandardScaler()
    df[atr] = ss.fit_transform(df[atr].values.reshape(-1, 1))

In [65]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 17 columns):
Age                         1470 non-null float64
Attrition                   1470 non-null int64
Department                  1470 non-null int64
DistanceFromHome            1470 non-null float64
Education                   1470 non-null float64
EducationField              1470 non-null int64
EnvironmentSatisfaction     1470 non-null float64
Gender                      1470 non-null int64
JobSatisfaction             1470 non-null float64
MaritalStatus               1470 non-null int64
MonthlyIncome               1470 non-null float64
OverTime                    1470 non-null int64
PerformanceRating           1470 non-null float64
RelationshipSatisfaction    1470 non-null float64
TotalWorkingYears           1470 non-null float64
YearsAtCompany              1470 non-null float64
YearsSinceLastPromotion     1470 non-null float64
dtypes: float64(11), int64(6)
memory usage: 195.3 KB


In [66]:
df.head()

,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion
0,0.446350,1,2,-1.010909,-0.891688,1,-0.660531,0,1.153254,2,-0.108350,1,-0.426230,-1.584178,-0.421642,-0.164613,-0.679146
1,1.322365,0,1,-0.147150,-1.868426,1,0.254625,1,-0.660853,1,-0.291719,0,2.346151,1.191438,-0.164511,0.488508,-0.368715
2,0.008343,1,1,-0.887515,-0.891688,4,1.169781,1,0.246200,2,-0.937654,1,-0.426230,-0.658973,-0.550208,-1.144294,-0.679146
3,-0.429664,0,1,-0.764121,1.061787,1,1.169781,0,0.246200,1,-0.763634,1,-0.426230,0.266233,-0.421642,0.161947,0.252146
4,-1.086676,0,1,-0.887515,-1.868426,3,-1.575686,1,-0.660853,1,-0.644858,0,-0.426230,1.191438,-0.678774,-0.817734,-0.058285


In [72]:
print('Breakdown of attrition.  (0) Stayed at company, (1) Left company')

df.Attrition.value_counts()

Breakdown of attrition.  (0) Stayed at company, (1) Left company


0    1233
1     237
Name: Attrition, dtype: int64

## Overview

This data set was created by IBM data scientists.  It describes 35 features for 1470 (fictional) employees including whether or not the employee left the firm (labeled "attrition" in the dataset).  Employees leave companies for a variety of reasons: disatisfaction with their role, their manager or their pay.  Perhaps they aren't necessarily dissatified with their current job but feel like something better is out there.  Or maybe they just feel like they'd been their long enough, and want something different.  Most likely its a combination of all of these things, plus a few others.  

Employers would like to have a sense of why and when an employee might leave.  If an employer believes that an employee that they really value might leave, they could respond and try to prevent them from leaving.  This is what we will attempt to predict using a wide and deep neural network.

### Evaluation Metric

Let's take a moment to break down what the three main metrics mean in our task, i.e. predicting whether or not an employee will leave a company, and what that mean for our businesses using the model:

* accuracy: Values getting true positives and negatives.  This probably isn't very useful for our dataset because there are significantly fewer people that left the company than stayed

* precision: Values a low false positive rate.  This probably isn't the best either, because while we wouldn't **want** to think that an employee is leaving when they aren't, it probably won't hurt the business, unless the employer grossly overreacts and scares them away

* recall: Values a low false negative rate.  This is the best metric for our case.  If our job is to see when employees leave, and if the fact is that they usually **don't** leave, and if its potentially pretty damaging to the firm when the employee **does** leave, we want to make sure that we miss as few cases as possible.

### Validation Method

Because of the imbalance in our prediction label we'll use a stratified split, this way we'll preserve the distribution in our model.  In an attempt to realistically generalize the overall performance of our model we'll use a nested cross-validation scheme.  The both the outer and inner loops will be 10-fold cross validations.  We use k-fold as opposed to a different cv scheme like shuffle-split, because our dataset is not that large and we would like to train on as much data as possible.  Using a k-fold cv ensures that we train on all of our data.  The inner loop will tune the hyper-parameters of our model which will be discussed later.

In [73]:
df

,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion
0,0.446350,1,2,-1.010909,-0.891688,1,-0.660531,0,1.153254,2,-0.108350,1,-0.426230,-1.584178,-0.421642,-0.164613,-0.679146
1,1.322365,0,1,-0.147150,-1.868426,1,0.254625,1,-0.660853,1,-0.291719,0,2.346151,1.191438,-0.164511,0.488508,-0.368715
2,0.008343,1,1,-0.887515,-0.891688,4,1.169781,1,0.246200,2,-0.937654,1,-0.426230,-0.658973,-0.550208,-1.144294,-0.679146
3,-0.429664,0,1,-0.764121,1.061787,1,1.169781,0,0.246200,1,-0.763634,1,-0.426230,0.266233,-0.421642,0.161947,0.252146
4,-1.086676,0,1,-0.887515,-1.868426,3,-1.575686,1,-0.660853,1,-0.644858,0,-0.426230,1.191438,-0.678774,-0.817734,-0.058285
5,-0.539166,0,1,-0.887515,-0.891688,1,1.169781,1,1.153254,2,-0.729850,0,-0.426230,0.266233,-0.421642,-0.001333,0.252146
6,2.417384,0,1,-0.764121,0.085049,3,0.254625,0,-1.567907,1,-0.814416,1,2.346151,-1.584178,0.092620,-0.981014,-0.679146
7,-0.758170,0,1,1.827158,-1.868426,1,1.169781,1,0.246200,0,-0.809529,0,2.346151,-0.658973,-1.321601,-0.981014,-0.679146
8,0.117845,0,1,1.703764,0.085049,1,1.169781,1,0.246200,2,0.642338,0,2.346151,-0.658973,-0.164511,0.325228,-0.368715
9,-0.101159,0,1,2.197341,0.085049,3,0.254625,1,0.246200,1,-0.268983,0,-0.426230,-0.658973,0.735447,-0.001333,1.493867


# this is just initial testing

In [100]:
from sklearn.model_selection import train_test_split

# stratified 90/10 train/test split`
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.Attrition)

y_train = df_train.Attrition
y_test = df_test.Attrition

X_train = df_train.drop('Attrition', axis=1).values
X_test = df_test.drop('Attrition', axis=1).values

print('train', X_train.shape, 'test', X_test.shape)

train (1323, 16) test (147, 16)


![keras](https://blog.keras.io/img/keras-tensorflow-logo.jpg)


# An example similar to Sklearn, Keras
We will start with creating a model that is similar to what we have seen in scikit-learn. Let's not worry about pre-processing the data right now. Let's just get familiar with Keras.

In [90]:
from sklearn import metrics as mt

In [91]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

Using TensorFlow backend.


In [92]:
# combine the features into a single large matrix
#X_train = df_train[feature_columns].values
#X_test = df_test[feature_columns].values

# This returns a tensor
inputs = Input(shape=(X_train.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

___
## Adding more customization
For this particular optimization, let's go beyond SGD and use a different optimizer. There are many excellent explanations of different optimizers, for instance:
- http://sebastianruder.com/optimizing-gradient-descent/

Now that we have defined the model, its as simple as using it in a very familiar syntax:

In [119]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [143]:
%%time

model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=1)

from sklearn import metrics as mt
yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

Epoch 1/100
1323/1323 [==============================] - 0s - loss: 0.3377 - acc: 0.8602     
Epoch 2/100
1323/1323 [==============================] - 0s - loss: 0.3135 - acc: 0.8753     
Epoch 3/100
1323/1323 [==============================] - 0s - loss: 0.3111 - acc: 0.8745     
Epoch 4/100
1323/1323 [==============================] - 0s - loss: 0.3097 - acc: 0.8730     
Epoch 5/100
1323/1323 [==============================] - 0s - loss: 0.3083 - acc: 0.8760     
Epoch 6/100
1323/1323 [==============================] - 0s - loss: 0.3078 - acc: 0.8760     
Epoch 7/100
1323/1323 [==============================] - 0s - loss: 0.3073 - acc: 0.8776     
Epoch 8/100
1323/1323 [==============================] - 0s - loss: 0.3062 - acc: 0.8791     
Epoch 9/100
1323/1323 [==============================] - 0s - loss: 0.3058 - acc: 0.8768     
Epoch 10/100
1323/1323 [==============================] - 0s - loss: 0.3054 - acc: 0.8791     
Epoch 11/100
1323/1323 [==============================] - 0

1323/1323 [==============================] - 0s - loss: 0.2972 - acc: 0.8753     
Epoch 88/100
1323/1323 [==============================] - 0s - loss: 0.2970 - acc: 0.8776     
Epoch 89/100
1323/1323 [==============================] - 0s - loss: 0.2970 - acc: 0.8776     
Epoch 90/100
1323/1323 [==============================] - 0s - loss: 0.2970 - acc: 0.8768     
Epoch 91/100
1323/1323 [==============================] - 0s - loss: 0.2969 - acc: 0.8760     
Epoch 92/100
1323/1323 [==============================] - 0s - loss: 0.2968 - acc: 0.8768     
Epoch 93/100
1323/1323 [==============================] - 0s - loss: 0.2970 - acc: 0.8768     
Epoch 94/100
1323/1323 [==============================] - 0s - loss: 0.2966 - acc: 0.8768     
Epoch 95/100
1323/1323 [==============================] - 0s - loss: 0.2966 - acc: 0.8768     
Epoch 96/100
1323/1323 [==============================] - 0s - loss: 0.2966 - acc: 0.8768     
Epoch 97/100
1323/1323 [==============================] - 0s - 

Well it didn't do very well.  It missed the majority of the true positives.  But there are a few issues here that we can address.

1. We don't have that much data to begin with and our target class is a small percentage of it so we're going to have a hard time.

2. We were using a mean squared error for our loss function on a binary classification task.  While this isn't terrible, it would likely work better if we used cross-entropy instead.

Luckily its easy to modify both of these in Keras.  The loss function is simple to change, and the fit function includes a parameter for 'class_weights' which accepts a dictionary of weights for each class value to use when computing the loss function.  This way we can tell the model which class is "more important".  Let's try it.

In [142]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=1, class_weight={0 : 0.20, 1 : 0.80})

yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

Epoch 1/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8337     
Epoch 2/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8314     
Epoch 3/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8299     
Epoch 4/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8330     
Epoch 5/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8330     
Epoch 6/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8292     
Epoch 7/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8330     
Epoch 8/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8330     
Epoch 9/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8262     
Epoch 10/100
1323/1323 [==============================] - 0s - loss: 0.1335 - acc: 0.8292     
Epoch 11/100
1323/1323 [==============================] - 0

1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8330     
Epoch 87/100
1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8367     
Epoch 88/100
1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8360     
Epoch 89/100
1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8352     
Epoch 90/100
1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8352     
Epoch 91/100
1323/1323 [==============================] - 0s - loss: 0.1326 - acc: 0.8345     
Epoch 92/100
1323/1323 [==============================] - 0s - loss: 0.1326 - acc: 0.8375     
Epoch 93/100
1323/1323 [==============================] - 0s - loss: 0.1327 - acc: 0.8352     
Epoch 94/100
1323/1323 [==============================] - 0s - loss: 0.1326 - acc: 0.8345     
Epoch 95/100
1323/1323 [==============================] - 0s - loss: 0.1326 - acc: 0.8352     
Epoch 96/100
1323/1323 [==============================] - 0s - 

It ended up getting a much better reacll score, at the expense of overall accuracy however.